# Day 11: Monkey in the Middle
https://adventofcode.com/2022/day/11

In [2]:
let lines: Vec<String> = std::fs::read_to_string("input/day11.txt")
    .expect("Could not read file")
    .lines()
    .map(|line| line.to_owned())
    .collect();

In [3]:
let example: Vec<String> = "Monkey 0:
  Starting items: 79, 98
  Operation: new = old * 19
  Test: divisible by 23
    If true: throw to monkey 2
    If false: throw to monkey 3

Monkey 1:
  Starting items: 54, 65, 75, 74
  Operation: new = old + 6
  Test: divisible by 19
    If true: throw to monkey 2
    If false: throw to monkey 0

Monkey 2:
  Starting items: 79, 60, 97
  Operation: new = old * old
  Test: divisible by 13
    If true: throw to monkey 1
    If false: throw to monkey 3

Monkey 3:
  Starting items: 74
  Operation: new = old + 3
  Test: divisible by 17
    If true: throw to monkey 0
    If false: throw to monkey 1".lines().map(|line| line.to_owned()).collect();

In [4]:
:dep regex = "1.7.0"

In [5]:
use regex::Regex;

In [6]:
struct Monkey {
    index: usize,
    operation: Box<dyn Fn(u32) -> u32>,
    test_divisible_by: u32,
    throw_to_monkey_if_true: usize,
    throw_to_monkey_if_false: usize
}

In [7]:
fn parse_operation(operation: &str) -> Box<dyn Fn(u32) -> u32> {
    if operation == "old * old" {
        return Box::new(|old| old * old);
    }
    
    let re = Regex::new(r"^old \+ (\d+)$").unwrap();
    if let Some(capture) = re.captures(&operation) {
        let n: u32 = capture.get(1).unwrap().as_str().parse().unwrap();
        return Box::new(move |old| old + n);
    }

    let re = Regex::new(r"^old \* (\d+)$").unwrap();
    if let Some(capture) = re.captures(&operation) {
        let n: u32 = capture.get(1).unwrap().as_str().parse().unwrap();
        return Box::new(move |old| old * n);
    }
    
    panic!("Could not parse operation {}", operation);
}

In [8]:
assert_eq!(49, parse_operation("old * old")(7));
assert_eq!(17, parse_operation("old + 8")(9));
assert_eq!(45, parse_operation("old * 5")(9));

In [9]:
// TODO: make method
fn parse_monkey(input: &String) -> (Monkey, Vec<u32>) {
    let re = Regex::new(r"^Monkey (?P<index>\d+):
  Starting items: (?P<starting_items>\d+(, \d+)*)
  Operation: new = (?P<operation_rhs>[old*+0-9 ]+)
  Test: divisible by (?P<test_divisible_by>\d+)
    If true: throw to monkey (?P<throw_to_monkey_if_true>\d+)
    If false: throw to monkey (?P<throw_to_monkey_if_false>\d+)$").unwrap();

    let caps = re.captures(&input).unwrap();
    
    let index = caps["index"].parse().unwrap();
    let items = caps["starting_items"].split(", ").map(|item| item.parse().unwrap()).collect();
    let operation = parse_operation(&caps["operation_rhs"]);
    let test_divisible_by = caps["test_divisible_by"].parse().unwrap();
    let throw_to_monkey_if_true = caps["throw_to_monkey_if_true"].parse().unwrap();
    let throw_to_monkey_if_false = caps["throw_to_monkey_if_false"].parse().unwrap();
    
    (Monkey {
        index,
        operation,
        test_divisible_by,
        throw_to_monkey_if_true,
        throw_to_monkey_if_false
    }, items)
}

In [10]:
fn parse_input(input: &[String]) -> (Vec<Monkey>, Vec<Vec<u32>>) {
    let monkeys_and_items = input.split(|line| line == "")
        .map(|block| parse_monkey(&block.join("\n")));
    
    let mut monkeys: Vec<Monkey> = Vec::new();
    let mut items_per_monkey: Vec<Vec<u32>> = Vec::new();
    
    for (monkey, items) in monkeys_and_items {
        monkeys.push(monkey);
        items_per_monkey.push(items);
    }

    (monkeys, items_per_monkey)
}

In [11]:
fn execute_monkey(monkey: &Monkey, items: Vec<Vec<u32>>) -> Vec<Vec<u32>> {
    let mut result: Vec<Vec<u32>> = items.clone();

    let monkey_items = &items[monkey.index];
    result[monkey.index].clear();

    for item in monkey_items {
        let new_item = (monkey.operation)(*item) / 3;
        let throw_to_monkey = if new_item % monkey.test_divisible_by == 0 {
            monkey.throw_to_monkey_if_true
        } else {
            monkey.throw_to_monkey_if_false
        };

        result[throw_to_monkey].push(new_item);
    }

    result
}

In [12]:
fn execute_round(monkeys: &Vec<Monkey>, items: Vec<Vec<u32>>) -> (Vec<Vec<u32>>, Vec<usize>) {
    let mut items = items;
    let mut inspection_count = Vec::<usize>::new();

    for monkey in &*monkeys {
        inspection_count.push(items[monkey.index].len());
        items = execute_monkey(monkey, items);
    }
    
    (items, inspection_count)
}

In [13]:
let (monkeys, items) = parse_input(&example);
execute_round(&monkeys, items)

([[20, 23, 27, 26], [2080, 25, 167, 207, 401, 1046], [], []], [2, 4, 3, 5])

In [18]:
use std::iter::successors;

fn count_inspections(monkeys: Vec<Monkey>, initial_items: Vec<Vec<u32>>, rounds: usize) -> Vec<usize> {
    // TODO: cloning might be wasteful here, but successors doesn't want me to move the argument
    successors(
        Some((initial_items, (0..monkeys.len()).map(|_| 0).collect())),
        |(items, _counts)| Some(execute_round(&monkeys, items.clone()))
    ).take(rounds + 1)
    .map(|(_items, counts)| counts)
    .reduce(|counts1, counts2| counts1.iter().zip(counts2.iter()).map(|(c1, c2)| c1 + c2).collect())
    .unwrap()
}

In [19]:
let (monkeys, initial_items) = parse_input(&example);
count_inspections(monkeys, initial_items, 20)

[101, 95, 7, 105]

In [28]:
fn part1(input: &[String]) -> usize {
    let (monkeys, initial_items) = parse_input(&input);
    let mut inspections_per_monkey = count_inspections(monkeys, initial_items, 20);
    inspections_per_monkey.sort();
    inspections_per_monkey.into_iter().rev().take(2).reduce(|a, b| a * b).unwrap()
}

In [29]:
part1(&example)

10605

In [30]:
part1(&lines)

90294

## Part 2

In [33]:
Regex::new(r"^foo (\d)
bar (\d)").unwrap().captures("foo 1
bar 2x").unwrap().get(1).unwrap().as_str()

"1"

In [35]:
let re = Regex::new(r"^old \+ (\d+)$").unwrap();
if let Some(capture) = re.captures("old + 5") {
    let n: u32 = capture.get(1).unwrap().as_str().parse().unwrap();
    println!("match: {}", n);
}

match: 5


()

In [ ]:
let re = Regex::new(r"^old + (\d+)$").unwrap();
if let Some(capture) = re.captures("old + r") {
    let n: u32 = capture.get(1).unwrap().as_str().parse().unwrap();
    println!("match: {}", n);
}